# Expected losses with Log Reg

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gaarutyunov/credit-risk/blob/master/notebooks/colab_log_reg_el.ipynb)

## Environment settings

For better performance change Colab runtime type to GPU

In [1]:
!git clone https://github.com/gaarutyunov/credit-risk.git

Cloning into 'credit-risk'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 151 (delta 75), reused 122 (delta 46), pack-reused 0
Receiving objects: 100% (151/151), 718.34 KiB | 22.45 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [2]:
%cd credit-risk

/content/credit-risk


In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/sberbank-ai/wing.git (to revision master) to /tmp/pip-install-9alfp8je/wing_c3df516b633e44f097ed3fd3c1df879a
  Running command git clone -q https://github.com/sberbank-ai/wing.git /tmp/pip-install-9alfp8je/wing_c3df516b633e44f097ed3fd3c1df879a
     |████████████████████████████████| 151 kB 28.1 MB/s 
     |████████████████████████████████| 79 kB 9.7 MB/s 
     |████████████████████████████████| 76.6 MB 1.8 MB/s 
     |████████████████████████████████| 155 kB 55.4 MB/s 
     |████████████████████████████████| 117 kB 56.0 MB/s 
     |████████████████████████████████| 596 kB 38.5 MB/s 
  Created wheel for wing: filename=wing-0.1.6-py3-none-any.whl size=12768 sha256=0cadc3fb3724310978ec3688da6f2b6ca37141e7a2f2abc7cd4a928d17f2c053
  Stored in directory: /tmp/pip-ephem-wheel-cache-vroomznh/wheels/d0/d3/78/d1cb5717341968f8319fa57765f68a894fc1b21d4f4068b519
  Created 

To get username and key follow instructions in [readme](https://github.com/Kaggle/kaggle-api)

In [ ]:
%env KAGGLE_USERNAME=<username>
%env KAGGLE_KEY=<key>

In [5]:
!kaggle datasets download wordsforthewise/lending-club

100% 1.26G/1.26G [00:10<00:00, 127MB/s]
100% 1.26G/1.26G [00:10<00:00, 135MB/s]


In [6]:
!unzip lending-club.zip

Archive:  lending-club.zip
  inflating: accepted_2007_to_2018Q4.csv.gz  
  inflating: accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv  
  inflating: rejected_2007_to_2018Q4.csv.gz  
  inflating: rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv  


In [7]:
!mkdir data

In [8]:
!mv accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv data/accepted_2007_to_2018Q4.csv
!mv rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv data/rejected_2007_to_2018Q4.csv

## Preprocessing

In [9]:
from pipeline import get_pipeline

preprocessing = get_pipeline(
    name="log_reg",
    group="preprocessing",
    debug=True,
)

_target_: pipeline.make_preprocessing_pipeline
steps_config:
- CSVReader:
    _target_: pipeline.CSVReader
    _convert_: all
    file: data/accepted_2007_to_2018Q4.csv
    columns:
    - loan_amnt
    - term
    - int_rate
    - emp_title
    - emp_length
    - home_ownership
    - annual_inc
    - verification_status
    - loan_status
    - purpose
    - addr_state
    - dti
    - earliest_cr_line
    - fico_range_high
    - inq_last_6mths
    - revol_bal
    - initial_list_status
    - out_prncp
    - total_rec_late_fee
    - collection_recovery_fee
    - last_fico_range_low
    - collections_12_mths_ex_med
    - application_type
    - tot_coll_amt
    - avg_cur_bal
    - bc_open_to_buy
    - chargeoff_within_12_mths
    - delinq_amnt
    - mo_sin_old_il_acct
    - mo_sin_old_rev_tl_op
    - mo_sin_rcnt_tl
    - mort_acc
    - mths_since_recent_bc
    - num_accts_ever_120_pd
    - num_actv_bc_tl
    - num_bc_tl
    - num_il_tl
    - num_sats
    - num_tl_120dpd_2m
    - num_tl_30dpd

In [10]:
X = preprocessing.fit_transform([], y=[])
y = preprocessing.label_transformer.label

/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:355: UserWarning: Persisting input arguments took 46.21s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name],


## Evaluation

In [ ]:
from pipeline import get_pipeline

prediction = get_pipeline(
    name="log_reg",
    group="prediction",
    debug=True,
)

In [ ]:
PD = prediction.predict_proba(X)

In [ ]:
LGD = 1.0

In [ ]:
from pipeline import get_pipeline

preprocessing = get_pipeline(
    name="log_reg",
    group="preprocessing",
    overrides=[
        "preprocessing_pipeline=raw_data"
    ],
    debug=True,
)

X_el = preprocessing.fit_transform([], y=[])

In [ ]:
EL = LGD * PD[:, 1] * X_el["funded_amnt"]

print(f"Expected losses: {EL:.2f}")